In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from HelperFunctions import *
from PtOccupancyFunctions import *
import pprint

In [ ]:
path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/SteffenDrt/09_03_2021/"

In [ ]:
def getDeltaMax(path):
    return path.split("/")[-1].split("_")[-1]

def getDirs(directory):
    result = []
    sub_dirs = []
    sdirs = [
        sdir.path
        for sdir in
            os.scandir(directory)
        if sdir.is_dir() and "DeltaMax" in sdir.name
    ]

    for sdir in sorted(sdirs, key=lambda x: float(getDeltaMax(x))):
        subresult = {}
        subresult["root"] = sdir
        for root, subdirs, files in os.walk(sdir):
            for file in files:
                if file=="0.trips.csv.gz" and "unimodal" not in root:
                    subresult["trips"] = os.path.join(root, file)
                if file=="0.drt_occupancy_time_profiles_drt.txt" and "unimodal" not in root:
                    subresult["drt_occupancy"] = os.path.join(root, file)
                if file=="trip_success.csv.gz" and "unimodal" not in root:
                    subresult["trip_success"] = os.path.join(root, file)
        result.append(subresult)
    
    return result

dirs = getDirs(path)
# pprint.pprint(dirs)

In [ ]:
def getTrips(paths, columns):
    path = paths["trips"]
    df = pd.read_csv(path, sep=";").loc[:,columns]
    df.set_index("person", inplace=True)
    return df

def getDrtOccupandyAndStandingFrac(paths, exclude_empty_vehicles):
    path = paths["drt_occupancy"]
    df = pd.read_csv(path, sep="\t")
    df["time"] = df["time"].apply(timestmphm2sec)
    drt_occ, drt_deviation = getAverageOcc(
        df.drop(columns="time"), exclude_empty_vehicles=exclude_empty_vehicles
    )
    drt_standing_frac = getStandingFraction(df.drop(columns="time"))
    return drt_occ, drt_standing_frac

def getTripSuccess(paths):
    path = paths["trip_success"]
    df = pd.read_csv(path, sep=";")
    return df

In [ ]:
delta_max_list = np.empty(15)
requested_dists = np.empty(15)
drt_occupancies_non_empty = np.empty(15)
drt_occupancies_non_standing = np.empty(15)
serviced_fracts = np.empty(15)

In [ ]:
columns=["person", "euclidean_distance"]

for i,dic in enumerate(dirs):
    delta_max_list[i] = getDeltaMax(dic["root"])
    print('Delta Max: ', delta_max_list[i])
    
    trips = getTrips(dic, columns)
    drt_occupancies_non_standing[i], _ = getDrtOccupandyAndStandingFrac(dic, False)
    drt_occupancies_non_empty[i], _ = getDrtOccupandyAndStandingFrac(dic, True)
    requested_dists[i] = trips['euclidean_distance'].mean()
    trip_succ = getTripSuccess(dic)
    serviced_fracts[i] = len(trip_succ[trip_succ["tripSuccess"]]) / len(trip_succ)
    

In [ ]:
data = np.stack((requested_dists, drt_occupancies_non_empty, drt_occupancies_non_standing, serviced_fracts))
columns=['MeanServicedEuclideanDistance', 'NonEmptyDrtOccupancy', 'NonStandingDrtOccupancy', 'ServicedFraction']
df = pd.DataFrame(data=data.T, index=delta_max_list, columns=columns)
df.index.rename('DeltaMax', inplace=True)

In [ ]:
display(df)
df.to_csv(os.path.join(path, 'DataAnalysis.csv'))